<a href="https://colab.research.google.com/github/kangwonlee/momisp/blob/dependabot/pip/tests/requests-2.32.0/Ch07_Stat.Indet/ex07.003.numpy.Double.Integral_bracket_fix_simple_v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.



`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)



In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비



## 예제 07.003<br>ex07.003



부정정보: 이중적분<br>Statically Indeterminate Beam : Double integration



p. 296



### 문제에서 주어진 변수<br>Given Parameters



#### 보의 길이<br>Length of the beam



In [ ]:
L_m = 4
a_m = 2



#### 하중<br>Load



In [ ]:
P_N = -5000



#### 잉여구속<br>Residual constraint



In [ ]:
delta_B_m = -30e-3



#### 재료와 단면 특성<br>Material and section properties



In [ ]:
E_Pa = 10e9
I_m4 = 20e6 * (1e-3) ** 4



#### 자유물체도<br>Free body diagram



In [ ]:
import os   # 운영체제 관련 기능 Operating Systems
import sys  # 시스템 관련 기능 Systems
# utils 폴더의 모듈을 import 할 수 있도록 준비
# add utils folder to sys.path to import
sys.path.append(os.path.abspath(os.path.join(os.pardir, 'utils')))
# 선도 관련 기능 diagrams
import draw_diagrams

points_list = [
    {'x_m': 0, 'text':'A'},
    {'x_m': L_m, 'text':'B'},
]

reaction_list = [
    {'x_m': 0},
    {'x_m': L_m},
]

dist_load_list = []

v_load_list = [
    {'x_m':a_m, 'sign': -1},
]

moment_list = [
    {'x_m': 0, 'direction': 'ccw', 'text': 'M', 'open':'right'},
]

draw_diagrams.draw_beam(L_m, points_list, reaction_list, v_load_list=v_load_list, dist_load_list=dist_load_list, moment_list=moment_list)



In [ ]:
x_m_array = np.arange(0, L_m + 0.5e-3, 1e-3)
x_A_m = 0
x_B_m = L_m



### 평형방정식<br>Equilibrium Equations



수직방향<br>Vertical direction



$$
R_A + R_B + P = 0
$$



모멘트방향 ($P<0$)<br>Moment direction ($P<0$)



$$
M_A - R_A \cdot L - P \cdot a = 0
$$



### 굽힘모멘트<br>Bending moment



$$
M(x) = EI\frac{d^2\nu}{dx^2} = -M_A +R_A x + P<x-a> [Nm]
$$



### 이중적분<br>Dual integration



$$
EI\frac{d\nu}{dx} = -M_A x +\frac{1}{2}R_A x^2 + \frac{1}{2}P<x-a>^2 + C_1 [Nm^2]
$$



$$
EI\nu = -\frac{1}{2}M_A x^2 +\frac{1}{6}R_A x^3 + \frac{1}{6}P<x-a>^3 + C_1 x + C_2 [Nm^3]
$$



### 경계조건<br>Boundary conditions



$$
EI\frac{d\nu}{dx}(x=0) = C_1 =0
$$



$$
EI\nu(x=0) = C_2 = 0 [Nm^3]
$$



또한 $\nu(x=L)=\delta$ 이므로<br>Additionally because $\nu(x=L)=\delta$



$$
\begin{align}
EI\nu(x=L) = EI\delta &= -\frac{1}{2}M_A L^2 +\frac{1}{6}R_A L^3 + \frac{1}{6}P<L-a>^3 \\
&= -\frac{1}{2}M_A L^2 +\frac{1}{6}R_A L^3 + \frac{1}{6}P(L-a)^3 \\
\end{align}
$$



양변에 6을 곱하면<br>Multiplying 6 to both sides gives



$$
6EI\delta= -3M_A L^2 +R_A L^3 + P(L-a)^3 
$$



평형방정식과 연립하면 다음과 같다.<br>Along with equilibrium equations:



$$
\begin{cases}
R_A + R_B + P = 0\\
M_A - R_A \cdot L - P \cdot a = 0\\
-3M_A L^2 +R_A L^3 + P(L-a)^3 = 6EI\delta
\end{cases}
$$



$$
\begin{cases}
R_A + R_B = - P\\
-M_A + R_A \cdot L = - P \cdot a\\
-3M_A L^2 +R_A L^3 = 6EI\delta - P(L-a)^3 
\end{cases}
$$



$$
\begin{bmatrix}
    0 & 1 & 1\\
    -1 & L & 0\\
    -3L^2 & L^3 & 0\\
\end{bmatrix}
\begin{pmatrix}
M_A \\ R_A \\ R_B
\end{pmatrix}
=
\begin{pmatrix}
-P \\ -Pa \\ 6EI\delta - P(L-a)^3
\end{pmatrix}
$$



In [ ]:
matA = np.matrix([
        (0, 1, 1),
        (-1, L_m, 0),
        (-3 * L_m ** 2, L_m ** 3, 0),
    ]
)



In [ ]:
matA



In [ ]:
vecB = np.matrix([
        (- P_N,),
        (- P_N * a_m,),
        (6 * E_Pa * I_m4 * delta_B_m - P_N * (L_m - a_m)**3,),
    ]
)



In [ ]:
vecB



In [ ]:
vecX = na.solve(matA, vecB)



In [ ]:
vecX



In [ ]:
M_A_sol_Nm, R_A_sol_N, R_B_sol_N = vecX.flatten().tolist()[0]



$$
\begin{align}
EI\nu(x=L) = EI\delta &= -\frac{1}{2}M_A L^2 +\frac{1}{6}R_A L^3 + \frac{1}{6}P<L-a>^3 \\
&= -\frac{1}{2}M_A L^2 +\frac{1}{6}R_A L^3 + \frac{1}{6}P(L-a)^3 \\
\end{align}
$$



In [ ]:
delta_mm_array = (-0.5 * M_A_sol_Nm * x_m_array**2 
    + 1/6. * R_A_sol_N * x_m_array**3 
    + 1/6. * P_N * (x_m_array - a_m) ** 3) / (E_Pa * I_m4) * 1e3



In [ ]:
plt.plot(x_m_array, delta_mm_array)
plt.grid(True)
plt.xlabel('$x(m)$')
plt.ylabel('$\\delta$(mm)')
plt.show()

